In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader

In [38]:
class CustomDataset(Dataset):
    def __init__(self,filepath,word2idx,tag2idx):
        self.word2idx = word2idx
        self.tag2idx = tag2idx
        self.sentences , self.ner_tags = self.load_data(filepath)

    def load_data(self,filepath):
        sentences, ner_tags = [],[]
        sentence, ner_tag = [],[]
        with open(filepath,"r",encoding = "utf-8") as f:
            for line in f:
                line = line.strip()
                if line:
                    word,_,ner = line.split("\t")
                    sentence.append(word)
                    ner_tag.append(ner)
                else:
                    if sentence:
                        sentences.append(sentence)
                        ner_tags.append(ner_tag)
                        sentence, ner_tag = [], []  # rest after every sentence
            return sentences,ner_tags
    def __len__(self):
      return len(self.sentences)

    def __getitem__(self, idx):
        word_ids = [self.word2idx.get(w, self.word2idx["<UNK>"]) for w in self.sentences[idx]]
        tag_ids = [self.tag2idx.get(t, self.tag2idx["<UNK>"]) for t in self.ner_tags[idx]]

        return word_ids, tag_ids

def collate_fn(batch):
  sentences,tag = zip(*batch)
  max_len = max(len(s) for s in sentences)
  padded_sentences = [  s+ [0] * (max_len-len(s)) for s in sentences]
  padded_tag = [ t + [0] *(max_len-len(t)) for t in tag]

  return torch.tensor(padded_sentences),torch.tensor(padded_tag)

Load Dataset

In [41]:
train_path = "/content/drive/MyDrive/Datasets/train_v5.conll"
val_path = "/content/drive/MyDrive/Datasets/val_v5.conll"
test_path = "/content/drive/MyDrive/Datasets/test_v5.conll"
word2idx = {"<PAD>": 0, "<UNK>": 1}
tag2idx = {"<PAD>": 0, "<UNK>": 1}
train_data = CustomDataset(train_path,word2idx, tag2idx)
val_data = CustomDataset(val_path,word2idx, tag2idx)
test_data = CustomDataset(test_path,word2idx, tag2idx)

train_load = DataLoader(train_data, batch_size=32, collate_fn=collate_fn)
val_load = DataLoader(val_data, batch_size=32, collate_fn=collate_fn)
test_load = DataLoader(test_data, batch_size=32, collate_fn=collate_fn)


In [44]:
for batch in train_load:
    sentences, tags = batch
    print("Sentences shape:", sentences.shape)
    print("Tags shape:", tags.shape)
    break


Sentences shape: torch.Size([32, 41])
Tags shape: torch.Size([32, 41])


In [ ]:
# pretrained with  fasttext HERE